# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
# Read in the data here
fname='../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")



In [3]:
df.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [4]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [47]:
df['i94yr'].drop_duplicates()

0    2016.0
Name: i94yr, dtype: float64

In [49]:
	
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql import 
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [27]:
#write to parquet
#df_spark.write.parquet("sas_data")
#df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [69]:
# Performing cleaning tasks here
df_img=df_spark[['cicid','arrdate','i94addr','visatype']]

In [70]:
#check for nulls 
df_img.where(col("i94addr").isNull()).count() #152592 null values

#df_img.where(col("cicid").isNull()).count()  # 0

#df_img.where(col("visatype").isNull()).count() 

152592

In [73]:
#dropping rows where i94addr is null because we want to do analysis by region so where region is not defined we don't want results there
df_imgnew=df_img.na.drop()

In [74]:
df_imgnew.where(col('i94addr').isNull()).count()

0

In [75]:
#converting epoch timestamp to date 
#df_spark.withColumn('arrdate',F.date_format(df_img.epoch.cast(dataType=t)))
df_imgnew=df_imgnew.withColumn('arrdate',from_unixtime(df_imgnew.arrdate/1000,"yyyy-MM-dd"))

In [76]:
#cleaned dataframe with 
df_imgnew.show()

+-----+----------+-------+--------+
|cicid|   arrdate|i94addr|visatype|
+-----+----------+-------+--------+
|  7.0|1970-01-01|     AL|      F1|
| 15.0|1970-01-01|     MI|      B2|
| 16.0|1970-01-01|     MA|      B2|
| 17.0|1970-01-01|     MA|      B2|
| 18.0|1970-01-01|     MI|      B1|
| 19.0|1970-01-01|     NJ|      B2|
| 20.0|1970-01-01|     NJ|      B2|
| 21.0|1970-01-01|     NY|      B2|
| 22.0|1970-01-01|     NY|      B1|
| 23.0|1970-01-01|     NY|      B2|
| 24.0|1970-01-01|     MO|      B2|
| 27.0|1970-01-01|     MA|      B1|
| 28.0|1970-01-01|     MA|      B1|
| 29.0|1970-01-01|     MA|      B2|
| 30.0|1970-01-01|     NJ|      B2|
| 31.0|1970-01-01|     NY|      B2|
| 33.0|1970-01-01|     TX|      B2|
| 34.0|1970-01-01|     CT|      B2|
| 35.0|1970-01-01|     CT|      B2|
| 36.0|1970-01-01|     NJ|      B2|
+-----+----------+-------+--------+
only showing top 20 rows



In [13]:
df_cities=pd.read_csv('us-cities-demographics.csv',delimiter=';')
df_cities.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [30]:
df_cities[df_cities['State']=='Massachusetts']

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
33,Framingham,Massachusetts,40.3,35442.0,35768.0,71210,2625.0,19070.0,2.49,MA,Black or African-American,6944
65,Somerville,Massachusetts,31.0,41028.0,39306.0,80334,2103.0,22292.0,2.43,MA,Asian,9498
85,Lynn,Massachusetts,34.8,44598.0,47861.0,92459,3093.0,29899.0,2.81,MA,Black or African-American,18240
86,Lynn,Massachusetts,34.8,44598.0,47861.0,92459,3093.0,29899.0,2.81,MA,Asian,9205
92,Newton,Massachusetts,42.3,41985.0,46824.0,88809,1814.0,21692.0,2.71,MA,Black or African-American,2925
119,Newton,Massachusetts,42.3,41985.0,46824.0,88809,1814.0,21692.0,2.71,MA,Hispanic or Latino,4790
129,Cambridge,Massachusetts,31.5,55421.0,54981.0,110402,2495.0,27757.0,2.06,MA,Asian,18441
167,Framingham,Massachusetts,40.3,35442.0,35768.0,71210,2625.0,19070.0,2.49,MA,Hispanic or Latino,13000
216,Brockton,Massachusetts,35.2,46273.0,49041.0,95314,3036.0,27313.0,2.88,MA,American Indian and Alaska Native,881


In [17]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = pd.read_csv(fname)


In [18]:
#df_temp['dt'].drop_duplicates()
df_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [21]:
df_temp[df_temp['Country']=='United States']

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W
47560,1820-06-01,25.682,2.008,Abilene,United States,32.95N,100.53W
47561,1820-07-01,26.268,1.802,Abilene,United States,32.95N,100.53W
47562,1820-08-01,25.048,1.895,Abilene,United States,32.95N,100.53W
47563,1820-09-01,22.435,2.216,Abilene,United States,32.95N,100.53W
47564,1820-10-01,15.830,2.169,Abilene,United States,32.95N,100.53W


In [15]:
df_air=pd.read_csv('airport-codes_csv.csv')
df_air.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [16]:
df_air['iso_region'].drop_duplicates()

0         US-PA
1         US-KS
2         US-AK
3         US-AL
4         US-AR
5         US-OK
6         US-AZ
7         US-CA
10        US-CO
11        US-FL
14        US-GA
16        US-HI
17        US-ID
19        US-IN
20        US-IL
24        US-KY
25        US-LA
28        US-MD
29        US-MI
30        US-MN
31        US-MO
32        US-MT
33        US-NJ
34        US-NC
36        US-NY
38        US-OH
41        US-OR
46        US-SC
47        US-SD
48        US-TX
          ...  
52095     MY-14
52100     MY-04
52107     MY-10
52210     TL-CO
52213     TL-BA
52215     TL-BO
52217     TL-VI
52618     VN-44
52623     VN-13
53470    AU-ACT
53949    CC-U-A
54019    CX-U-A
54137    NF-U-A
54556      PY-9
54691    ZA-U-A
54755     ME-21
54863    ZM-U-A
54872    MN-055
54873    MN-065
54874    MN-039
54875    MN-069
54876    MN-067
54877    MN-053
54878    MN-043
54879    MN-051
54880    MN-061
54890    MN-063
54891    MN-073
54893      MN-1
54894    MN-046
Name: iso_region, Length

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.